In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ddfdfdf/trials_xgb_gbtree_file.pkl


In [2]:
!pip install gdown
import gdown
print(f'downloading the datasets and data dictionary: ')
data_url = 'https://drive.google.com/uc?export=download&confirm=6gh6&id=1VNpyNkGxHdskfdTNRSjjyNa5qC9u0JyV'
output_data = "fraud_data.csv"
data_dict_url = 'https://drive.google.com/uc?id=1VQ-HAm0oHbv0GmDKP2iqqFNc5aI91OLn&export=download'
output_dict = "data_dict.txt"
gdown.download(data_dict_url, output_dict, quiet=False)
gdown.download(data_url, output_data, quiet=False)

downloading the datasets and data dictionary: 


Downloading...
From: https://drive.google.com/uc?id=1VQ-HAm0oHbv0GmDKP2iqqFNc5aI91OLn&export=download
To: /kaggle/working/data_dict.txt
100%|██████████| 1.30k/1.30k [00:00<00:00, 2.06MB/s]
Downloading...
From: https://drive.google.com/uc?export=download&confirm=6gh6&id=1VNpyNkGxHdskfdTNRSjjyNa5qC9u0JyV
To: /kaggle/working/fraud_data.csv
100%|██████████| 494M/494M [00:08<00:00, 59.0MB/s]


'fraud_data.csv'

In [3]:
import pandas as pd

print(f'reading the data into pandas dataframe: ')
df = pd.read_csv('/kaggle/working/fraud_data.csv')

reading the data into pandas dataframe: 


In [4]:
from sklearn.model_selection import StratifiedShuffleSplit

print(f'cleaning and encoding the features in the dataframe: ')
df_encoded = pd.get_dummies(df, columns=['type'])
df_encoded[df_encoded.select_dtypes(include=['bool']).columns] = df_encoded.select_dtypes(include=['bool']).astype(int)
df_final = df_encoded.drop(['nameOrig', 'nameDest'], axis=1)
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
X = df_final.drop('isFraud', axis=1)
y = df_final['isFraud']
print(f'splitting the dataset into train and test datasets: ')
for train_index, test_index in split.split(X, y):
  X_train, X_test = X.iloc[train_index], X.iloc[test_index]
  y_train, y_test = y.iloc[train_index], y.iloc[test_index]
X_train.shape, X_test.shape, y_train.shape, y_test.shape

cleaning and encoding the features in the dataframe: 
splitting the dataset into train and test datasets: 


((5090096, 12), (1272524, 12), (5090096,), (1272524,))

In [5]:
# import pickle

# with open('/kaggle/input/ddfdfdf/trials_xgb_gbtree_file.pkl', 'rb') as f:
#     trials_resume = pickle.load(f)


In [6]:
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer, roc_auc_score, f1_score, r2_score, precision_score, recall_score
from hyperopt import fmin, hp, tpe, Trials
import pickle

print('starting hyperparameter tuning')
sss2 = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

def objective(params):
    # print(f'params: {params}')
    model = XGBClassifier(random_state=42, booster='gbtree', device='cuda', **params)
    scores = cross_val_score(model, X_train_scaled, y_train, cv=sss2, scoring=make_scorer(recall_score), n_jobs=-1)
    mean_scores = np.mean(scores)
    return -mean_scores

space = {
    
    'learning_rate':hp.loguniform('learning_rate', 0.0001, 0.3),
    'n_estimators':hp.uniformint('n_estimators', 60, 600),
    'max_depth':hp.uniformint('max_depth', 3, 10),
    'gamma':hp.uniformint('gamma', 0, 5),
    'subsample':hp.uniform('subsample', 0.5, 1.0),
    'lambda':hp.uniform('lambda', 0.0, 1.0),
    'alpha':hp.uniform('alpha', 0.0, 1.0),
    
}

trials_xgb_gbtree_recall_score = Trials()
best = fmin(fn=objective, space=space, algo=tpe.suggest, trials=trials_xgb_gbtree_recall_score, max_evals=50, verbose=1)


trials_file_name = "trials_xgb_gbtree_recall_score.pkl"
print(f'downloading the trials object using pickle to the file named:')

with open(trials_file_name, 'wb') as f:
    pickle.dump(trials_xgb_gbtree_recall_score, f)

starting hyperparameter tuning
  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [19:33:28] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)
/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [19:33:29] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for boo

100%|██████████| 50/50 [57:46<00:00, 69.34s/trial, best loss: -0.8004566210045662]
downloading the trials object using pickle to the file named:


In [7]:
# trials_file_name = "trials_xgb_gbtree_file.pkl"
# print(f'downloading the trials object using pickle to the file named:')

# with open(trials_file_name, 'wb') as f:
#     pickle.dump(trials_xgb_gbtree, f)


In [8]:
# best_trial = trials_resume.best_trial
print(f'extracting best hyperparams from trials object: ')
best_trial = trials_xgb_gbtree_recall_score.best_trial
print(best_trial['result']['loss'])
best_hyperparams = {k:v[0] for k, v in best_trial['misc']['vals'].items()}
for key in ('max_depth', 'n_estimators'):
    best_hyperparams[key] = int(best_hyperparams[key])
best_hyperparams

extracting best hyperparams from trials object: 
-0.8004566210045662


{'alpha': 0.6803984100840382,
 'gamma': 4.0,
 'lambda': 0.7238022737664775,
 'learning_rate': 1.012646328868626,
 'max_depth': 5,
 'n_estimators': 153,
 'subsample': 0.7184163850866028}

In [9]:
from xgboost import XGBClassifier
import xgboost as xgb
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

final_model = XGBClassifier(random_state=42, verbosity=2, booster='gbtree', device='cuda', **best_hyperparams)
print(f'fitting the train data with the best hyperparams found from hyperopt.')
final_model.fit(X_train_scaled, y_train)

fitting the train data with the best hyperparams found from hyperopt.


XGBClassifier(alpha=0.6803984100840382, base_score=None, booster='gbtree',
              callbacks=None, colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device='cuda', early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=4.0, grow_policy=None, importance_type=None,
              interaction_constraints=None, lambda=0.7238022737664775,
              learning_rate=1.012646328868626, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=153, n_jobs=None, ...)

In [10]:
from sklearn.metrics import make_scorer, roc_auc_score, f1_score, r2_score, precision_score, recall_score
import pandas as pd

print(f'Calculating the scores on the test set: ')
X_test_scaled = scaler.transform(X_test)
y_pred = final_model.predict(X_test_scaled)

columns = ['f1', 'roc_auc',  'precision', 'recall']
scores = [f1_score(y_test, y_pred), roc_auc_score(y_test, y_pred),  precision_score(y_test, y_pred), recall_score(y_test, y_pred)]
res_df = pd.DataFrame([scores], columns=columns)
res_df.head()

Calculating the scores on the test set: 


/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [20:30:24] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


,f1,roc_auc,precision,recall
0,0.107092,0.545549,0.128292,0.091905


In [11]:
print(f'saving the results df into the file: results_recall_score_tuned.csv')
res_df.to_csv('results_recall_score_tuned.csv', index=False)
print(f'DONE')

saving the results df into the file: results_recall_score_tuned.csv
DONE
